Google Colab is cool

In [1]:
!nvidia-smi

Mon Mar 31 11:25:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Check current versions
!python -c "import numpy; print('Current numpy version:', numpy.__version__)"
!python -c "import torch; print('Current torch version:', torch.__version__)"

# Install desired versions with CUDA 12.4
!pip install numpy==1.26.4
!pip install torch==2.3.0+cu124 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.3.0+cu124.html
!pip install torcheeg
!pip install pytorch-lightning

# This will show the versions in new Python processes
!python -c "import numpy; print('Installed numpy version:', numpy.__version__)"
!python -c "import torch; print('Installed torch version:', torch.__version__)"

print("\n*** IMPORTANT: Please restart the runtime now for changes to take effect in this notebook ***")

Current numpy version: 2.0.2
Current torch version: 2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==2.3.0+cu124 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")

if torch.cuda.is_available():
    accelerator = "gpu"
    pin_memory = True
    print("GPU is available. Training will use GPU acceleration.")
else:
    accelerator = "cpu"
    pin_memory = False
    print("WARNING: GPU not available, using CPU instead")

PyTorch version: 2.6.0+cu124
CUDA version: 12.4
GPU is available. Training will use GPU acceleration.


In [2]:
from torcheeg.datasets import DREAMERDataset
from torcheeg import transforms
from torcheeg.datasets.constants import DREAMER_CHANNEL_LOCATION_DICT
from torcheeg.model_selection import KFoldGroupbyTrial
from torcheeg.models import CCNN
from torcheeg.trainers import ClassifierTrainer

from torch.utils.data import DataLoader, random_split
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import os

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# !cp /content/drive/MyDrive/datasets/DREAMER/DREAMER.mat /content/DREAMER.mat
# os.mkdir("/content/model")
# IO_PATH = '/content/io/'
# MODEL_SAVE_PATH = '/content/model/'
# MAT_FILE_PATH = '/content/DREAMER.mat'

IO_PATH = '/content/drive/MyDrive/datasets/DREAMER/io/'
MODEL_SAVE_PATH = '/content/drive/MyDrive/datasets/DREAMER/model/'
MAT_FILE_PATH = '/content/drive/MyDrive/datasets/DREAMER/DREAMER.mat'


# if os.path.exists(IO_PATH):
#     raise Exception("io folder ins't allowed to exist")

if not os.path.exists(MODEL_SAVE_PATH):
    raise Exception("make sure model folder exists")

if not os.path.exists(MAT_FILE_PATH):
    raise Exception("make sure MAT file exists")


SEED:int = 42
BATCH_SIZE:int = 64
N_WORKERS:int = 4   # rule of thumb: n_workers = 4 * n_gpus

In [12]:
# https://zenodo.org/records/546113
# Katsigiannis, S., & Ramzan, N. (2017). DREAMER: A Database for Emotion Recognition through EEG and ECG Signals from Wireless Low-cost Off-the-Shelf Devices [Data set]. Zenodo. https://doi.org/10.1109/JBHI.2017.2688239
# we predict arousal (intensity of emotion) and valence (negativity of emotion) on scale of 1-5, negative and aroused means fear (or masochism (me fr))

# if this is throwing an error about corrupted, make sure the folder is
# populated by the right stuff (from running this thing the last time)
# or does NOT exist
dataset = DREAMERDataset(
    io_path=IO_PATH,
    mat_path=MAT_FILE_PATH,

    offline_transform=transforms.Compose([
        transforms.BandDifferentialEntropy(apply_to_baseline=True),
        transforms.ToGrid(DREAMER_CHANNEL_LOCATION_DICT, apply_to_baseline=True) # grid will be important later in training
    ]),

    online_transform=transforms.Compose([
        transforms.BaselineRemoval(), #
        transforms.ToTensor()
    ]),

    # valence should be smaller than 3 (negative emotions), arousal should be higher than 3
    # here this will return an in [0, 1, 2, 3] for each possible combination of the two binaries
    # for us it is fear, when [0, 1] (so 1)
    label_transform=transforms.Compose([
        transforms.Select(['valence', 'arousal']),
        transforms.Binary(3),
        transforms.BinariesToCategory()
    ]),

    num_worker=N_WORKERS
)

# will populate io_path - the next time this is run, we won't take 55 minutes, since
# all we need to do are the online_transforms

[2025-03-31 12:13:35] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from /content/drive/MyDrive/datasets/DREAMER/io/.
INFO:torcheeg:🔍 | Detected cached processing results, reading cache from /content/drive/MyDrive/datasets/DREAMER/io/.


In [16]:
# validation is used during training for evaluating each batch
# test is saved to after (even if we don't train on val anyway, mechanisms like picking the best may still have 'val bias')
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"Dataset split: {train_size} training, {val_size} validation, {test_size} test samples")



# when training a GPU, we put the memory on a special 'pinned' region for faster transfer between CPU and GPU
# we shuffle the data in training to avoid it learning some patterns there (across batches) - no need on eval, since metrics will be the same
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=N_WORKERS,
    pin_memory=pin_memory
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=pin_memory
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=pin_memory
)


# ([64, 4, 9, 9]) - ([batch_size, n_bands, grid_dim, grid_dim])
for batch in train_loader:
    print(f"Input shape: {batch[0].shape}")
    break

Dataset split: 60020 training, 12861 validation, 12863 test samples


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Input shape: torch.Size([64, 4, 9, 9])


In [19]:
model = CCNN(
    num_classes=4,  # Binary classification (fear vs. non-fear)
    in_channels=4,  # 4 frequency bands
    grid_size=(9, 9)    # channels mapped to a grid (not every one has an electrode) - each frequency band has its own grid, similar to how we have multiple values for RGB
)


callbacks = [
    # saves the currently model to specific file
    ModelCheckpoint(
        dirpath=MODEL_SAVE_PATH,
        filename='best-model-{epoch:02d}-{val_accuracy:.4f}',
        monitor='val_accuracy',
        mode='max',
        save_top_k=1
    ),

    # saves the last model to specific file
    ModelCheckpoint(
        dirpath=MODEL_SAVE_PATH,
        filename='last-model',
        save_last=True
    ),

    # tries to detect overfitting by stopping if the val_accuracy stops improving for 10 epochs
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max',
        verbose=True
    )
]


# wrapper over pl.LightningModule
# TODO
# - precision="16-mixed" # T4 gpu works well for this (isn't supported here)
trainer = ClassifierTrainer(
    model=model,
    num_classes=4,
    lr=1e-4,
    weight_decay=1e-4,
    accelerator=accelerator,
)

In [20]:
print("\nStarting model training...")
trainer.fit(
    train_loader,
    val_loader,
    max_epochs=100,
    default_root_dir=MODEL_SAVE_PATH,
    callbacks=callbacks,
    enable_progress_bar=True,
    enable_model_summary=True
)
# ends once the trainer is done (so most likely through EarlyStop, otherwise max_epochs)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | CCNN             | 6.2 M  | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | train
6 | val_metrics   | MetricCollection | 0      | train
7 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
6.2 M     Trainabl


Starting model training...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[2025-03-31 12:20:06] INFO (torcheeg/MainThread) 
[Val] val_loss: 1.397 val_accuracy: 0.156 
INFO:torcheeg:
[Val] val_loss: 1.397 val_accuracy: 0.156 


Training: |          | 0/? [00:00<?, ?it/s]

...

[2025-03-31 13:06:40] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.913 val_accuracy: 0.664 
INFO:torcheeg:
[Val] val_loss: 0.913 val_accuracy: 0.664 
[2025-03-31 13:06:40] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.517 train_accuracy: 0.789 
INFO:torcheeg:
[Train] train_loss: 0.517 train_accuracy: 0.789 


Validation: |          | 0/? [00:00<?, ?it/s]

[2025-03-31 13:07:08] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.935 val_accuracy: 0.663 
INFO:torcheeg:
[Val] val_loss: 0.935 val_accuracy: 0.663 
[2025-03-31 13:07:08] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.510 train_accuracy: 0.793 
INFO:torcheeg:
[Train] train_loss: 0.510 train_accuracy: 0.793 


Validation: |          | 0/? [00:00<?, ?it/s]

[2025-03-31 13:07:35] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.942 val_accuracy: 0.666 
INFO:torcheeg:
[Val] val_loss: 0.942 val_accuracy: 0.666 
[2025-03-31 13:07:35] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.503 train_accuracy: 0.798 
INFO:torcheeg:
[Train] train_loss: 0.503 train_accuracy: 0.798 
INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_accuracy did not improve in the last 10 records. Best score: 0.667. Signaling Trainer to stop.


In [26]:
print("\nEvaluating model on test set...")
test_results = trainer.test(test_loader)[0]
print(test_results)
print(f"Test accuracy: {test_results['test_accuracy']:.4f}")

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Evaluating model on test set...


Testing: |          | 0/? [00:00<?, ?it/s]

[2025-03-31 13:21:46] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.935 test_accuracy: 0.665 
INFO:torcheeg:
[Test] test_loss: 0.935 test_accuracy: 0.665 


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6649304032325745     │
│         test_loss         │    0.9354316592216492     │
└───────────────────────────┴───────────────────────────┘

{'test_loss': 0.9354316592216492, 'test_accuracy': 0.6649304032325745}
Test accuracy: 0.6649


2025-03-31 13:26:06.720209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743427566.741048   34596 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743427566.747453   34596 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
